<a href="https://colab.research.google.com/github/martin23-sg/Procesos-Estocasticos/blob/main/Ruina_del_Jugador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **El Problema de la Ruina del Jugador (Simulación del Juego)**

**Objetivo:** Realizar múltiples simulaciones del juego de la Ruina del Jugador y verificar sí se aproximan la probabilidad empírica y duración empírica del juego a las teóricas obtenidas en clase.

**Enunciado del Problema:** Sean $k$ y $a$ dos números enteros tales que $1 \le k < a$ y sea $p \in (0,1)$. Supongamos que se tienen dos jugadores, digamos un jugador $A$ y un jugador $B$, con capital inicial de $k$ unidades monetarias y $a-k$ unidades monetarias, respectivamente, (de modo que, hay $a$ unidades monetarias en total en juego). Estos jugadores participan en un juego en el que, en cada ronda, alguno de ellos cede una unidad monetaria al otro en caso de perder dicha ronda. De esta manera, supongamos también que, el jugador $A$ tiene una probabilidad $p$ de ganar cada ronda, el jugador $B$ una probabilidad $q := 1-p$ de ganar cada ronda y que el resultado obtenido en cada una de las rondas es independiente de los obtenidos en las demás rondas. El juego termina cuando alguno de los dos jugadores se encuentre arruinado, es decir, una vez que su capital llega a $0$ unidades monetarias.



In [229]:
# Librerias a usar
import random
import numpy as np
import pandas as pd

In [230]:
## Parámetros (puedes cambiar alguno de ellos para ver qué pasa, los siguientes son sólo para un ejemplo ilustrativo):

n_sims = 10000 # Cantidad de simulaciones del juego

dinero_A = 100 # Cantidad de dinero con que inicia el jugador A

dinero_B = 10 # Cantidad de dinero con que inicia el jugador B

p = 0.45 # Probabilidad del jugador A de ganar cada ronda

In [231]:
q = 1-p # Probabilidad del jugador B de ganar cada ronda

## **Probabilidad de ruina del jugador $A$ y duración del juego (teóricas)**

Ahora, centrémonos en el jugador $A$. Puede verifcarse que, de forma teórica, la probabilidad de ruina del jugador $A$ y la duración del juego, denotadas por $u_k$ y $d_k$ respectivamente, están dadas por:
$$
u_k =
\begin{cases}
\frac{s^k-s^a}{1-s^a} & \text{ si } s \neq 1 \\
\frac{a-k}{a}         & \text{ si } s = 1
\end{cases}
$$

$$
d_k =
\begin{cases}
\frac{1}{1-2p}(k-\frac{a(1-s^k)}{1-s^a}) & \text{ si } s \neq 1 \\
k(a-k)                                   & \text{ si } s = 1
\end{cases}
$$
**Nota:** $s:=\frac{q}{p} =\frac{1-p}{p}$.

In [232]:
# Cálculos teóricos con las expresiones anteriores:
k = dinero_A # Capital incial de A
a = dinero_A + dinero_B # Total de capital en juego

s = q/p

if s!=1:
  u_k = (s**k - s**a)/(1-s**a)
  d_k = 1/(1-2*p)*(k-a*(1-s**k)/(1-s**a))
else:
  u_k = (a-k)/a
  d_k = k*(a-k)

In [233]:
print(f'La probabilidad teórica de ruina del jugador A es: {round(u_k, 4)}')
print(f'La duración teórica del juego es: {round(d_k, 4)} rondas')

La probabilidad teórica de ruina del jugador A es: 0.8656
La duración teórica del juego es: 852.1263 rondas


## **Simulaciones del juego**

In [234]:
def simualciones_juego(n_sims, dinero_A, dinero_B):
  '''
  Función para realizar múltiples simulaciones del juego de la Ruina del Jugador devolviendo el número de rondas en cada simulación,
  así como el número de veces que se arruina el jugador A en el total de simulaciones.
  Imputs:
  n_sims: Cantidad de simulaciones del juego
  dinero_A: Cantidad de dinero con que inicia el jugador A
  dinero_B: Cantidad de dinero con que inicia el jugador B
  Outputs:
  rondas: Arreglo con el número de rondas de cada simulación del juego
  ruinas_de_A: Número de veces que se arruina A en las simulaciones del juego
  '''
  rondas = np.zeros(n_sims) # Arreglo para guardar el número de rondas de cada simulación del juego
  ruinas_de_A = 0 # Contador para el número de veces que se arruina A

  ## Simulaciones del juego
  for i in range(n_sims):
    A = dinero_A # Capital inicial de A
    B = dinero_B # Capital inicial de B
    num_rondas = 0 # Contador para el número de rondas en un juego
    # Ejecución del juego
    while (A!=0 and B!=0): # Mientras que ninguno de los jugadores esté arruinado, el juego continua
      # Ejecución de una ronda
      resultado = random.choices([1,-1], weights=[p,q], k=1)[0] # Con probabilidad p, A gana y con probabilidad q A pierde (y en consecuencia, B gana)
      # Si A gana la ronda, el dinero de A se aumenta en una unidad y el de B disminuye una unidad. Ocurre visceversa si A pierde la ronda
      A += resultado
      B -= resultado
      # Termina la ronda
      num_rondas += 1 # Se aumenta en una unidad el número de rondas del juego
      # Si el jugador A se arruina en esta ronda, se aumenta en una unidad las veces que se arruinó A
      if A == 0:
        ruinas_de_A += 1
      ## Termina el juego
    rondas[i] = num_rondas # Se guarda el número de rondas de este juego
  ## Finalización de las simulaciones del juego
  return rondas, ruinas_de_A

In [235]:
# Ejecución de las simulaciones del juego mediante la función anterior
rondas, ruinas_de_A = simualciones_juego(n_sims, dinero_A, dinero_B)

#### **Estimación de la Probabilidad de ruina de $A$ y de la Duración del juego**

In [236]:
print(f'La probabilidad empírica de ruina del jugador A es: {round(ruinas_de_A/n_sims, 4)}')
print(f'La duración empírica del juego es: {round(rondas.mean(), 4)} rondas')

La probabilidad empírica de ruina del jugador A es: 0.8617
La duración empírica del juego es: 849.1774 rondas


#### **Comparación de resultados**

In [237]:
tabla = {
    "Resultado": ["Teórico", "Vía simulaciones"],
    "Probabilidad de ruina de A": [u_k, round(ruinas_de_A/n_sims, 4)],
    "Duración del juego (número de rondas)": [d_k, round(rondas.mean(), 4)]
}
resultados = pd.DataFrame(tabla)
resultados

,Resultado,Probabilidad de ruina de A,Duración del juego (número de rondas)
0,Teórico,0.865569,852.126304
1,Vía simulaciones,0.861700,849.177400


**Conclusión:** Al menos con este ejemplo ilustrativo (e incluso si movemos los valores de los parámetros del programa), podemos ver que los resultados muestrales o empíricos vía simulación se aproximan de buena manera a los resultados teóricos esperados.